# This is Cartography put together into one place. 

## Modules to add to run this code:

- [BioPython][1]
- [Pandas][2]
- [Numpy][3]
- [Altair][4]
- [Seaborn][5]
- [Scikit-Learn][6]
- [UMAP][7]
- json
- nextstrain-augur
[1]:https://biopython.org/wiki/Download
[2]:https://pandas.pydata.org/pandas-docs/version/0.23.3/install.html
[3]:https://docs.scipy.org/doc/numpy/user/quickstart.html
[4]:https://altair-viz.github.io/getting_started/installation.html
[5]:https://seaborn.pydata.org/installing.html
[6]:https://scikit-learn.org/stable/install.html
[7]:https://umap-learn.readthedocs.io/en/latest/


# Imports Section 

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.spatial.distance import squareform, pdist
import pandas as pd
import numpy as np
from Bio import SeqIO
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from augur.utils import json_to_tree
import json
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
import umap
import statistics
from scipy.stats import linregress
from pathlib import Path
from Helpers import *

### Reading in the Fasta File
- I used BioPython to parse the Fasta file into two numpy Arrays: Genomes and Strains. 

In [ ]:
dropped_strains = ["Yunnan/01/2019"]

In [ ]:
len(dropped_strains)

In [ ]:
#work on making this work
path = "../zika-nextstrain/results/aligned.fasta"
strains = []
genomes = []

for record in SeqIO.parse(path, "fasta"):
    if(record.id not in dropped_strains):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

#### Checking to make sure the file I picked is a aligned Fasta file / is the file I wanted

In [ ]:
len(strains)
print(len(genomes[0]) == len(genomes[1]))
print(len(genomes))

In [ ]:
strains[:5]

# Creating the Distance Matrix
- I used Hamming Distance to find the pairwise distance between each genome and each other genome, effectively creating a similarity/distance matrix
    - In my Hamming Distance method, I only counted something as a different if it was a mismatch between the nucleotides (A,G,C, or T), not gaps (as that was throwing off the algorithm too much for smaller strains)
- I then used Seaborn to generate a heatmap to make sure the matrix looked correct

In [ ]:
distance_matrix_file = "DistanceMatrixZika.csv"

In [ ]:
%%time
# Try to load an existing distance matrix. Create it, if it doesn't already exist.
try:
    # The index should be the first column and correspond to strain name for the row.
    similarity_matrix = pd.read_csv(distance_matrix_file, index_col=0)
    print("Loaded existing distance matrix")
except FileNotFoundError:
    print("Could not find existing distance matrix, creating it now", end="...")
    
    # Calculate Hamming distances.
    hamming_distances = get_hamming_distances(genomes)
    
    # Convert distinct pairwise distances into the more redundant but more interpretable square matrix.
    similarity_matrix = squareform(hamming_distances)
    
    # Convert the numpy matrix to a pandas data frame with strain annotations for rows and columns.
    similarity_matrix = pd.DataFrame(
        similarity_matrix,
        columns=strains,
        index=strains
    )
    
    # Write out the resulting data frame to cache distance calculations.
    # Keep the index in the output file, so it is immediately available on read.
    similarity_matrix.to_csv(distance_matrix_file)
    print("done!")

In [ ]:
print(sns.heatmap(similarity_matrix))

# Reading in the Metadata
- The metadata is used for getting the region, country, etc of different strains. This data is used to color the clusters.
- The metadata contains all of the possible sampled strains, so many of these will not be in the genomes from the aligned file, probably because the strains were corrupted, too short, etc. 
- We merge this metadata with the strains we have in the aligned file to get a list of all of the strains that match between both. It should come out to the amount of strains in the aligned file.

In [ ]:
#merging my final dataframe with their regions and strain names
cwd = Path.cwd()
goal_dir = cwd.parent
pathMeta = goal_dir.joinpath("zika-nextstrain", "results", "metadata.tsv")
metadata_df = pd.read_csv(pathMeta, delimiter='\t')

In [ ]:
metadata_df.head()

In [ ]:
metadata_df.shape

In [ ]:
#making a dataframe out of the strains from the alignment file to merge with the metadata
strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

In [ ]:
strains_df.shape

In [ ]:
#Merging strains and metadata 
result_strains = pd.merge(strains_df,metadata_df, on='strain', how= "left")

In [ ]:
#checking that no strains were lost
result_strains.shape

In [ ]:
np.setdiff1d(strains_df['strain'].unique(),metadata_df['strain'].unique())

# Creating the Phylogenetic Tree in Altair
- I used Altair to make this tree (Documentation linked [here][1]
- I opened and imported the json from a build from NextStrain ([flu][2], [zika][3], etc)
- The data from the JSON and the Data from the tree are usually a little different, so after merging the two dataframes you may get some errors.

[1]: https://altair-viz.github.io/index.html
[2]: https://github.com/nextstrain/seasonal-flu
[3]: https://altair-viz.github.io/index.html

In [ ]:
with open('../zika-nextstrain/auspice/zika-cartography_tree.json') as fh:
    json_tree_handle = json.load(fh)

In [ ]:
tree = json_to_tree(json_tree_handle)

In [ ]:
tree

In [ ]:
node_data = [
    {
        "strain": node.name,
        "date": node.attr["num_date"],
        "y": node.yvalue,
        "region": node.attr["region"],
        "country": node.attr["country"],
        "parent_date": node.parent is not None and node.parent.attr["num_date"] or node.attr["num_date"],
        "parent_y": node.parent is not None and node.parent.yvalue or node.yvalue,
        "clade_membership" : node.attr['clade_membership']
    }
    for node in tree.find_clades(terminal=True)
]

In [ ]:
node_data[10]

In [ ]:
node_df = pd.DataFrame(node_data)

In [ ]:
node_df.head()

In [ ]:
node_df["y"] = node_df["y"].max() - node_df["y"]

In [ ]:
node_df["parent_y"] = node_df["parent_y"].max() - node_df["parent_y"]

In [ ]:
node_df.shape

In [ ]:
node_df.head()

In [ ]:
node_df["region"].unique()

In [ ]:
node_numpy = node_df['clade_membership'].to_numpy()
clade_ok = []
for node in node_numpy:
    clade_ok.append(node)

In [ ]:
clade_new = pd.DataFrame(clade_ok)
node_df = node_df.merge(clade_new, how='outer', left_index=True, right_index=True)

In [ ]:
node_df.head()

In [ ]:
node_df.columns = ['strain', 'date', 'y', 'region', 'country', 'parent_date',
      'parent_y', 'clade_membership', 'clade_membership_color']

## Checking for Outliers in Pairwise Distance

In [ ]:
result_df = pd.DataFrame(result_strains['strain'])
similarity_matrix.shape

In [ ]:
index_df = pd.DataFrame(similarity_matrix.mean().index)

In [ ]:
result_index_df = result_df.set_index(similarity_matrix.mean().index)
mean_similarity_matrix = pd.DataFrame(similarity_matrix.mean())

In [ ]:
result_index_df = result_index_df.merge(mean_similarity_matrix, left_index=True, right_index=True)
result_index_df.columns = ["strain","mean"]
result_index_df

In [ ]:
alt.Chart(result_index_df,height=150).mark_boxplot().encode(
    x = alt.X('mean',title="mean of pairwise distances"),
    tooltip = ["strain"]
)

In [ ]:
genomes_missing_bases = []
for x in genomes:
    x = re.sub(r'[^AGCT]', '5', x)
    numberOfN = x.count("5")
    genomes_missing_bases.append(numberOfN)

In [ ]:
genomes_missing_bases_df = pd.DataFrame(genomes_missing_bases)
genomes_missing_bases_df = genomes_missing_bases_df.merge(strains_df, how='outer', left_index = True, right_index = True)
genomes_missing_bases_df.columns = ["bases missing", "strain"]

# Missing Bases

In [ ]:
genomes_missing_bases_df.head()

In [ ]:
dropped_strains

In [ ]:
alt.Chart(genomes_missing_bases_df).mark_bar().encode(
    alt.X("bases missing:Q", bin=True),
    y="count()"
)

In [ ]:
# TODO: extend dropped strains to omit those with lots of missing bases.
# Also, consider storing the list of dropped strains in a text file outside of the notebook.
#dropped_strains.extend(list(genomes_missing_bases_df[genomes_missing_bases_df["bases missing"] > 1000]["strain"].values))

# Running PCA on Scaled and Centered Data
- I treated each nucleotide as a "site", or dimension, and found the probability of having a certain nucleotide given the frequency of that letter at that site.
- I used [this paper][1] as my source 
- The equation is as follows where C is the matrix of dimensions, M is the mean, and p is the frequency of a nucleotide at that given site. 
![](https://journals.plos.org/plosgenetics/article/file?type=thumbnail&id=info:doi/10.1371/journal.pgen.0020190.e003)

In [ ]:
numbers = genomes[:]
for i in range(0,len(genomes)):
    numbers[i] = re.sub(r'[^AGCT]', '5', numbers[i])
    numbers[i] = list(numbers[i].replace('A','1').replace('G','2').replace('C', '3').replace('T','4'))
    numbers[i] = [int(j) for j in numbers[i]]
genomes_df = pd.DataFrame(numbers)
genomes_df.columns = ["Site " + str(k) for k in range(0,len(numbers[i]))]

In [ ]:
genomes_df.head()

In [ ]:
#performing PCA on my pandas dataframe 
pca = PCA(n_components=10,svd_solver='full') #can specify n, since with no prior knowledge, I use None
principalComponents = pca.fit_transform(genomes_df)
principalDf = pd.DataFrame(data = principalComponents, columns = ["PCA" + str(i) for i in range(1,11)])

In [ ]:
df = pd.concat([pd.DataFrame(np.arange(1,11)), pd.DataFrame([round(pca.explained_variance_ratio_[i],4) for i in range(0,len(pca.explained_variance_ratio_))])], axis = 1)
df.columns = ['principal components','explained variance']
df

In [ ]:
alt.Chart(df).mark_point().encode(
    x='principal components:Q',
    y='explained variance:Q')

In [ ]:
principalDf.head()

# Merging Strain and PCA

In [ ]:
strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

In [ ]:
result_strains = pd.merge(strains_df,metadata_df, on='strain')

In [ ]:
finalDf = pd.concat([principalDf, result_strains[['region','country','strain']]], axis = 1)

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain","region"]).dropna()

In [ ]:
merged_df.head()

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain","region"]).dropna()

In [ ]:
merged_df.head()

In [ ]:
list_of_chart = linking_tree_with_plots_brush(merged_df,['PCA1','PCA2','PCA3','PCA4'],
                                         ['Principal Component 1 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[0]*100,2)) + ")",
                                          'Principal Component 2 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[1]*100,2)) + ")",
                                          'PCA3 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[2]*100,2)) + ")",
                                          'PCA4 (Explained Variance : {}%'.format(round(pca.explained_variance_ratio_[3]*100,2)) + ")"],
                                         "clade_membership:N",['strain','region'])
chart = list_of_chart[0]|list_of_chart[1]|list_of_chart[2]
chart.save("../Docs/PCAFluBrush.html")
chart

In [ ]:
PCA_violin_df = get_euclidean_data_frame(merged_df, "PCA1", "PCA2", "PCA")
g = sns.FacetGrid(
    PCA_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

# Running MDS on the Dataset

In [ ]:
embedding = MDS(n_components=10,metric=True,dissimilarity='precomputed')
X_transformed = embedding.fit_transform(similarity_matrix)

In [ ]:
raw_stress = embedding.stress_
normalized_stress = np.sqrt(raw_stress /((similarity_matrix.values.ravel() ** 2).sum() / 2))
print(normalized_stress.round(2))

In [ ]:
MDS_df = pd.DataFrame(X_transformed,columns=['MDS' + str(i) for i in range(1,11)])
print(MDS_df.head())

In [ ]:
MDS_df.set_index(similarity_matrix.index)
finalDf = concatenate_results_with_strain_data(MDS_df, node_df, ['strain','clade_membership', 'clade_membership_color'])

In [ ]:
finalDf.columns

In [ ]:
finalDf.head()

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain",'clade_membership_color','clade_membership']).dropna()

In [ ]:
chart_12_mds = scatterplot_with_tooltip_interactive(merged_df,'MDS1','MDS2',"MDS1","MDS2",['strain','clade_membership'],'clade_membership_color')
chart_34_mds = scatterplot_with_tooltip_interactive(merged_df,'MDS3','MDS4',"MDS3","MDS4",['strain','clade_membership'],'clade_membership_color')
chart_56_mds = scatterplot_with_tooltip_interactive(merged_df,'MDS5','MDS6',"MDS5","MDS6",['strain','clade_membership'],'clade_membership_color')
chart_12_mds|chart_34_mds|chart_56_mds

In [ ]:
chart_12_scatter = scatterplot_tooltips(strains, similarity_matrix, merged_df, "MDS1", "MDS2", "MDS", 4000)
chart_34_scatter = scatterplot_tooltips(strains, similarity_matrix,merged_df, "MDS3", "MDS4", "MDS", 4000)
chart_56_scatter = scatterplot_tooltips(strains, similarity_matrix,merged_df, "MDS5", "MDS6", "MDS", 4000)
chart_12_scatter | chart_34_scatter | chart_56_scatter

In [ ]:
MDS_violin_df = get_euclidean_data_frame(merged_df, "MDS1", "MDS2", "MDS")
g = sns.FacetGrid(
    MDS_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

In [ ]:
LOESS_scatterplot(strains, similarity_matrix, merged_df, "MDS1", "MDS2", "MDS", 4000)

# Linking Tree to MDS plot

In [ ]:
list_of_data_and_titles = ['MDS1','MDS2','MDS3','MDS4','MDS5','MDS6']
list_of_chart = linking_tree_with_plots_brush(merged_df,list_of_data_and_titles,list_of_data_and_titles,'clade_membership_color',["clade_membership","strain:N"])
chart = list_of_chart[0]|list_of_chart[1]|list_of_chart[2]|list_of_chart[3]
chart

In [ ]:
chart.save("../Docs/MDSFluBrush.html")

# Running T-SNE on the Dataset 

In [ ]:
embedding = TSNE(n_components=2,metric='precomputed',perplexity = 25.95)
X_transformed = embedding.fit_transform(similarity_matrix)

In [ ]:
TSNE_df = pd.DataFrame(X_transformed,columns=['TSNE' + str(i) for i in range(1,3)])
print(TSNE_df.head())

In [ ]:
finalDf = concatenate_results_with_strain_data(TSNE_df,result_strains,['strain','region'])

In [ ]:
finalDf.shape

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain"]).dropna()

In [ ]:
merged_df.columns

In [ ]:
chart = scatterplot_tooltips(strains, similarity_matrix, merged_df, "TSNE1", "TSNE2", "TSNE", 4000)
chart

In [ ]:
scatterplot_with_tooltip_interactive(merged_df,'TSNE1','TSNE2',"TSNE1","TSNE2",['strain','clade_membership'],'clade_membership_color')

# Linking Tree to T-SNE plot

In [ ]:
list_of_chart = linking_tree_with_plots_brush(merged_df,['TSNE1','TSNE2'],['TSNE1','TSNE2'],'clade_membership_color',["clade_membership:N","strain:N"])
chart = list_of_chart[0]|list_of_chart[1]
chart

In [ ]:
TSNE_violin_df = get_euclidean_data_frame(merged_df, "TSNE1", "TSNE2", "TSNE")
g = sns.FacetGrid(
    TSNE_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

In [ ]:
LOESS_scatterplot(strains, similarity_matrix, merged_df, "TSNE1", "TSNE2", "TSNE", 4000)

# Running UMAP on the Dataset

In [ ]:
reducer = umap.UMAP(n_neighbors=20,
        min_dist=1.0,
        n_components=2,
        init="spectral")
embedding = reducer.fit_transform(similarity_matrix)

In [ ]:
UMAP_df = pd.DataFrame(embedding,columns=['UMAP' + str(i) for i in range(1,3)])
print(UMAP_df.head())

In [ ]:
finalDf = concatenate_results_with_strain_data(UMAP_df, result_strains, ['strain','region','country'])

In [ ]:
finalDf.shape

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain","region"]).dropna()

In [ ]:
merged_df.shape

In [ ]:
chart = scatterplot_tooltips(strains, similarity_matrix, merged_df, "UMAP1", "UMAP2", "UMAP", 4000)
chart

In [ ]:
scatterplot_with_tooltip_interactive(merged_df,'UMAP1','UMAP2',"UMAP1","UMAP2",['strain','clade_membership'],'clade_membership_color')

# Linking Tree with UMAP plot

In [ ]:
list_of_data_and_titles = ['UMAP1','UMAP2']
list_of_chart = linking_tree_with_plots_brush(merged_df,list_of_data_and_titles,list_of_data_and_titles,'clade_membership_color',["clade_membership","strain:N"])
chart = list_of_chart[0]|list_of_chart[1]
chart

In [ ]:
chart.save("../Docs/UMAPFluBrush.html")

In [ ]:
UMAP_violin_df = get_euclidean_data_frame(merged_df, "UMAP1", "UMAP2", "UMAP")
g = sns.FacetGrid(
    UMAP_violin_df,
    col="embedding",
    col_wrap=3,
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

In [ ]:
LOESS_scatterplot(strains, similarity_matrix, merged_df, "UMAP1", "UMAP2", "UMAP", 4000)

# Linking all plots together clickable with Tree

In [ ]:
together_df = MDS_df.merge(TSNE_df, how='outer', left_index=True, right_index=True)
together_df = together_df.merge(principalDf, how='outer', left_index=True, right_index=True)
together_df = together_df.merge(UMAP_df, how='outer', left_index=True, right_index=True)
together_df.head()

In [ ]:
finalDf = concatenate_results_with_strain_data(together_df, result_strains, ['strain','region','country'])

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain","region"]).dropna()

In [ ]:
merged_df.columns

In [ ]:
data = linking_tree_with_plots_clickable(merged_df,['MDS1', 'MDS2','TSNE1', 'TSNE2', 'PCA1', 'PCA2', 'UMAP1', 'UMAP2'], ['MDS1', 'MDS2', 'TSNE1', 'TSNE2','Principal Component 1 (Expected Variance : {}%'.format(round(pca.explained_variance_ratio_[0]*100,2)) + ")",
'Principal Component 2 (Expected Variance : {}%'.format(round(pca.explained_variance_ratio_[1]*100,2)) + ")",'UMAP1','UMAP2'],'clade_membership_color:N',['clade_membership'],['strain','clade_membership'])

In [ ]:
PCAMDS = data[3]|data[1]|data[5]
TSNEUMAP = data[2]|data[4]
embeddings = alt.vconcat(PCAMDS,TSNEUMAP)
embeddings
fullChart = alt.hconcat(data[0],embeddings)
fullChart.save('../Docs/FullLinkedChartClickable.html')
fullChart

In [ ]:
data_frames = [
    scatterplot_xyvalues(strains, similarity_matrix, merged_df, "PCA1", "PCA2", "PCA"),
    scatterplot_xyvalues(strains, similarity_matrix, merged_df, "MDS1", "MDS2", "MDS"),
    scatterplot_xyvalues(strains, similarity_matrix, merged_df, "TSNE1", "TSNE2", "TSNE"),
    scatterplot_xyvalues(strains, similarity_matrix, merged_df, "UMAP1", "UMAP2", "UMAP"),
]

In [ ]:
merged_index = np.array(sorted(merged_df.index.values))

In [ ]:
genetic_distances = squareform(similarity_matrix.values[merged_index][:, merged_index])

In [ ]:
data_frames.append(pd.DataFrame({
    "distance": genetic_distances,
    "embedding":"genetic"
}))

In [ ]:
len(data_frames)

In [ ]:
euclidean_data_frame = pd.concat(data_frames, sort=False)

In [ ]:
euclidean_data_frame.head()

In [ ]:
euclidean_data_frame.shape

In [ ]:
PCA = scatterplot_tooltips(strains, similarity_matrix, merged_df, "PCA1", "PCA2", "PCA", 4000)
MDS = scatterplot_tooltips(strains, similarity_matrix, merged_df, "MDS1", "MDS2", "MDS", 4000)
TSNE = scatterplot_tooltips(strains, similarity_matrix, merged_df, "TSNE1", "TSNE2", "TSNE", 4000)
UMAP = scatterplot_tooltips(strains, similarity_matrix, merged_df, "UMAP1", "UMAP2", "UMAP", 4000)

chart = PCA|MDS|TSNE|UMAP
chart.save('../Docs/FullScatterplot.html')

In [ ]:
chart

# Notes to Self:

- Collapse cells underneath Markdown headers
- Get docstrings above methods to show up when user presses SHIFT + TAB
- link back to the methods section for user each time method is used
- Run more Flu builds (3 years, 6 years, 12 years, H3, H1)
- Try algorithm on MERS
- Try algorithm on other bacterial genomes (unaligned / small snips of genomes)
- Make Zika clades or run automatic clade naming on 12y H3N2 flu with different cutoffs to standardize coloring for graphs
- Write a paper


## Within- and between-clade Euclidean distances for all embeddings

Use the complete embedding data frame to calculate pairwise Euclidean distances between samples and plot the results in a single figure.

In [ ]:
data_frames = [
    get_euclidean_data_frame(merged_df, "PCA1", "PCA2", "PCA"),
    get_euclidean_data_frame(merged_df, "MDS1", "MDS2", "MDS"),
    get_euclidean_data_frame(merged_df, "TSNE1", "TSNE2", "t-SNE"),
    get_euclidean_data_frame(merged_df, "UMAP1", "UMAP2", "UMAP"),
]

In [ ]:
data_frames

Extract pairwise genetic (Hamming) distances corresponding to the records sampled above. This step assumes that the original merged data frame is indexed from zero to N for N total samples in the same order as the similarity matrix.

In [ ]:
genetic_distances = squareform(similarity_matrix)

In [ ]:
data_frames.append(pd.DataFrame({
    "distance": genetic_distances,
    "clade_status": data_frames[0]["clade_status"].values,
    "embedding": "genetic"
}))

In [ ]:
len(data_frames)

In [ ]:
euclidean_data_frame = pd.concat(data_frames)

In [ ]:
g = sns.FacetGrid(
    euclidean_data_frame,
    col="embedding",
    col_wrap=3,
    col_order=["PCA", "MDS", "t-SNE", "UMAP"],
    sharey=False,
    height=4
)
g = g.map(sns.violinplot, "clade_status", "distance", order=["within", "between"])
g.set_axis_labels("Clade status", "Distance")

plt.savefig("../docs/FullViolinPlot.png")

In [ ]:
PCA_df = euclidean_data_frame[euclidean_data_frame.embedding == "PCA"]
MDS_df = euclidean_data_frame[euclidean_data_frame.embedding == "MDS"]
TSNE_df = euclidean_data_frame[euclidean_data_frame.embedding == "t-SNE"]
UMAP_df = euclidean_data_frame[euclidean_data_frame.embedding == "UMAP"]
genetic_df = euclidean_data_frame[euclidean_data_frame.embedding == "genetic"]

In [ ]:
genetic = alt.Chart(genetic_df,width=300).mark_boxplot().encode(
    x="clade_status:N",
    y="distance:Q"
).properties(title="genetic")
PCA = alt.Chart(PCA_df,width=200).mark_boxplot().encode(
    x="clade_status:N",
    y="distance:Q"
).properties(title="PCA")
MDS = alt.Chart(MDS_df,width=200).mark_boxplot().encode(
    x="clade_status:N",
    y="distance:Q"
).properties(title="MDS",width=200)
TSNE = alt.Chart(TSNE_df).mark_boxplot().encode(
    x="clade_status:N",
    y="distance:Q"
).properties(title="TSNE",width=200)
UMAP = alt.Chart(UMAP_df,width=200).mark_boxplot().encode(
    x="clade_status:N",
    y="distance:Q"
).properties(title="UMAP")

chart = genetic|PCA|MDS|TSNE|UMAP
chart

In [ ]:
chart.save('../Docs/FullBoxplot.html')

In [ ]:
#ratio of within vs between (within shorter than between) - label which metrics + sginificance level

In [ ]:
genetic_df.mean()

In [ ]:
median_genetic_within = genetic_df[genetic_df.clade_status == "within"].median()
median_genetic_between = genetic_df[genetic_df.clade_status == "between"].median()

In [ ]:
median_PCA_within = PCA_df[PCA_df.clade_status == "within"].median()
median_PCA_between = PCA_df[PCA_df.clade_status == "between"].median()

In [ ]:
median_MDS_within = MDS_df[MDS_df.clade_status == "within"].median()
median_MDS_between = MDS_df[MDS_df.clade_status == "between"].median()

In [ ]:
median_TSNE_within = TSNE_df[TSNE_df.clade_status == "within"].median()
median_TSNE_between = TSNE_df[TSNE_df.clade_status == "between"].median()

In [ ]:
median_UMAP_within = UMAP_df[UMAP_df.clade_status == "within"].median()
median_UMAP_between = UMAP_df[UMAP_df.clade_status == "between"].median()

In [ ]:
def ratioFunction(num1, num2):
    ratio12 = int(num1/num2)
    return ratio12

In [ ]:
genetic_ratio = ratioFunction(median_genetic_between,median_genetic_within)
print(genetic_ratio)

In [ ]:
PCA_ratio = ratioFunction(median_PCA_between,median_PCA_within)
print(PCA_ratio)

In [ ]:
MDS_ratio = ratioFunction(median_MDS_between,median_MDS_within)
print(MDS_ratio)

In [ ]:
TSNE_ratio = ratioFunction(median_TSNE_between,median_TSNE_within)
print(TSNE_ratio)

In [ ]:
UMAP_ratio = ratioFunction(median_UMAP_between,median_UMAP_within)
print(UMAP_ratio)